# NLP Project 
##### Santiago Martin & Léo Ringeissen

## Data loading

In [42]:
import pandas as pd
offerings = pd.read_csv('data/offerings.csv',sep=',',header=0)
reviews = pd.read_csv('data/reviews.csv',sep=',',header=0)

In [43]:
offerings.head()

,hotel_class,region_id,url,phone,details,address,type,id,name
0,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,"{'region': 'NY', 'street-address': '147 West 4...",hotel,113317,Casablanca Hotel Times Square
1,5.0,32655,http://www.tripadvisor.com/Hotel_Review-g32655...,NaN,NaN,"{'region': 'CA', 'street-address': '300 S Dohe...",hotel,76049,Four Seasons Hotel Los Angeles at Beverly Hills
2,3.5,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,"{'region': 'NY', 'street-address': '790 Eighth...",hotel,99352,Hilton Garden Inn Times Square
3,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,"{'region': 'NY', 'street-address': '152 West 5...",hotel,93589,The Michelangelo Hotel
4,4.0,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,"{'region': 'NY', 'street-address': '130 West 4...",hotel,217616,The Muse Hotel New York


In [44]:
reviews.head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False


## Data Preprocessing

#### Drop useless columns

In [46]:
cols_to_drop_offerings = ['hotel_class', 'region_id', 'url', 'phone', 'details', 'address', 'type']
offerings = offerings.drop(cols_to_drop_offerings, axis=1)

cols_to_drop_review = ['author', 'date_stayed', 'num_helpful_votes', 'date', 'id', 'via_mobile']
reviews = reviews.drop(cols_to_drop_review, axis=1)

#### Create rating columns

In [ ]:
import pandas as pd
import ast

# Define the desired categories
categories = ["service", "cleanliness", "overall", "value", "location", "sleep_quality", "rooms"]

# Function to parse the ratings column
def extract_ratings(row):
    try:
        rating_dict = ast.literal_eval(row)  # Convert string to dictionary
        if all(cat in rating_dict for cat in categories):  # Check if all required categories are present
            return {cat: rating_dict.get(cat) for cat in categories}  # Extract only the required categories
    except (ValueError, SyntaxError):
        return None  # Return None for rows with invalid or incomplete data
    return None

# Process the DataFrame
def process_reviews_dataframe(df):
    # Parse the ratings column and extract relevant categories
    df["parsed_ratings"] = df["ratings"].apply(extract_ratings)

    # Drop rows where any required category is missing
    df = df.dropna(subset=["parsed_ratings"])

    # Expand the parsed ratings into separate columns
    ratings_df = pd.json_normalize(df["parsed_ratings"])
    df = pd.concat([df.drop(columns=["ratings", "parsed_ratings"]), ratings_df], axis=1)

    return df

# Example usage
# Replace 'your_dataframe' with your actual DataFrame
reviews = process_reviews_dataframe(reviews)

# Display or save the processed DataFrame
reviews.head()

,title,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,93338.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",93338.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,“Great Stay”,This is a great property in Midtown. We two di...,1762573.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,1762573.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,1762573.0,4.0,5.0,4.0,3.0,5.0,5.0,5.0


In [55]:
reviews.dropna(inplace=True)
reviews['offering_id'] = reviews['offering_id'].astype(int)

#### Joining offerings and reviews

In [56]:
# Merge the reviews and offerings dataframes on the offering_id and id columns
merged_df = pd.merge(reviews, offerings, left_on='offering_id', right_on='id')

# Group by the hotel id and name, and calculate the mean for the rating columns and count for the number of reviews
grouped_df = merged_df.groupby(['id', 'name']).agg(
    {cat: 'mean' for cat in categories} | {'offering_id': 'count'}
).rename(columns={'offering_id': 'num_reviews'}).reset_index()

# Display the resulting dataframe
grouped_df.head()

,id,name,service,cleanliness,overall,value,location,sleep_quality,rooms,num_reviews
0,73445,BEST WESTERN PLUS Westchase Mini-Suites,4.132530,4.313253,4.084337,4.060241,4.349398,3.963855,4.048193,83
1,73463,Downtowner Inn and Suites,2.857143,3.714286,3.000000,3.000000,4.428571,3.285714,3.428571,7
2,73470,Greenway Inn & Suites,4.000000,4.250000,3.750000,3.875000,4.500000,4.250000,3.750000,8
3,73481,Holiday Inn Express - Houston,5.000000,4.600000,4.800000,4.400000,4.800000,4.800000,4.600000,5
4,74363,BEST WESTERN PLUS Independence Park Hotel,4.084848,4.078788,3.757576,3.787879,4.254545,3.975758,3.793939,165


In [57]:
grouped_df.shape

(1298, 10)

In [59]:
print(grouped_df['num_reviews'].sum())

212659
